# Libs

In [29]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from enum import Enum
from IPython.display import HTML
from typing import Callable

SEED = 50
np.random.seed(SEED)

%matplotlib notebook

# Map

In [34]:
class ItemType(Enum):
    EMPTY = 0
    OBSTACLE = 3
    TARGET = 2
    START = 1

class Map:
    def __init__(self, shape: tuple[int, int] = (10, 10), resolution: int = 1, name: str = 'Map') -> None:
        self.shape = np.array(shape) * resolution
        self.map = np.zeros(self.shape)
        self.name = name
    
    def draw(self, ax: plt.Axes):
        ax.set_title(self.name)
        ax.grid(visible=True, which='both', axis='both', color='gray', linestyle='--')
        # ax.set_xticks(np.arange(0, 10, 1))
        # ax.set_yticks(np.arange(0, 10, 1))
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

        return ax.imshow(self.map, cmap="Blues", interpolation='nearest')

    def reset_map(self) -> None:
        self.map = np.zeros(self.shape)

    def add_item(self, x: int, y: int, value: ItemType = ItemType.OBSTACLE) -> None:
        if x < 0 or y < 0 or x >= self.shape[0] or y >= self.shape[1]:
            return
        
        if self.map[y, x] != ItemType.EMPTY.value:
            return

        self.map[y, x] = value.value

    def set_goal(self, x: int, y: int) -> None:
        self.add_item(x, y, ItemType.TARGET)
        self.goal = np.array((x, y))

    def set_start(self, x: int, y: int) -> None:
        self.add_item(x, y, ItemType.START)

    def set_square_wall(self, x: int, y: int, length: int, height: int) -> None:
        self.add_item(x, y, ItemType.OBSTACLE)
        for i in range(length):
            for j in range(height):
                self.add_item(x+i, y+j)
    
    def has_obstacle(self, x: int, y: int) -> bool:
        if x < 0 or y < 0 or x >= self.shape[0] or y >= self.shape[1]:
            return True
        
        return self.map[y, x] == ItemType.OBSTACLE.value
    
    def has_target(self, x: int, y: int) -> bool:
        return self.map[y, x] == ItemType.TARGET.value

    def __str__(self) -> str:
        return str(self.map)
    

# Agent

In [51]:
class Agent:
    def __init__(self, intial_pos: tuple[float, float] = (0, 0), initial_velocity: tuple[float, float] = (0, 0),
                 weight: float = 0.5, c1: float = 0.5, c2: float = 0.5, id = 0, map_shape: tuple[int, int] = (10, 10)) -> None:
        # Current position and speed
        self.X = np.array(intial_pos, dtype=np.float16)
        self.V = np.array(initial_velocity, dtype=np.float16)

        # Inertial weight
        self.w = weight

        # Personal and social constants
        self.c1 = c1
        self.c2 = c2

        # Personal best position
        self.pbest = np.random.rand(2)

        # Cost function
        self.cost = None

        # Simulation parameters
        self.id = id
        self.map_shape = map_shape
        self.target_reached = False

    
    def get_euclidean_distance(self, a: tuple[int, int], b: tuple[int, int]) -> float:
        """Calculates the euclidean distance between two points

        Args:
            a (tuple[int, int]): First point
            b (tuple[int, int]): Second point

        Returns:
            float: Euclidean distance
        """
        return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    
    def has_exceeded_limits(self, position: tuple[int, int], has_obstacle: Callable[[int, int], bool] = None) -> bool:
        """Checks if the agent has exceeded the limits of the map or some obstacle"""
        if position[0] < 0 or position[1] < 0 or position[0] >= self.map_shape[0] or position[1] >= self.map_shape[1]:
            return True
        
        if has_obstacle is not None and has_obstacle(int(position[0]), int(position[1])):
            return True
        
        return False

    def update_position(self, has_obstacle: Callable[[int, int], bool] = None,
                        has_target: Callable[[int, int], bool] = None) -> None:
        next_pos = self.X + self.V

        if self.has_exceeded_limits(next_pos, has_obstacle):
            print(f"Agent {self.id} hit an obstacle")
             # Reduce velocity
            self.V = 0.5 * self.V
            # Or change randomly the direction TODO
            self.V = np.random.rand(2) * self.V
        elif has_target is not None and has_target(int(next_pos[0]), int(next_pos[1])):
            print(f"Agent {self.id} reached the target")
            self.X = next_pos
            self.V = 0.01 * self.V
            self.target_reached = True
        else:
            self.X = next_pos
  
    def update_velocity(self, gbest: tuple[int, int]) -> None:
        # Randomly set r1 and r2
        r1 = np.random.rand()
        r2 = np.random.rand()
        self.V = self.w * self.V + self.c1 * r1 * (self.pbest - self.X) + self.c2 * r2 * (gbest - self.X)

    def update_pbest(self, signal_distance: float, global_cost: float, gbest: tuple[int, int]) -> None:
        # Handle first assignment of agent cost
        if self.cost is None:
            self.cost = signal_distance
            self.pbest = self.X
            # Handle first assignment of global cost
            if global_cost is None:
                global_cost = self.cost
                gbest = self.X
            # Handle global best improvement in the first agent cost assignment
            elif self.cost < global_cost:
                gbest = self.X
                global_cost = self.cost
        # Handle agent cost improvement
        elif signal_distance < self.cost:
            self.pbest = self.X
            self.cost = signal_distance
            # print(f"Agent {self.id} improved. Cost: {self.cost:.2f} Pos: {self.pbest}")
            # Handle global best improvement
            if self.cost < global_cost:
                gbest = self.X
                global_cost = self.cost
                # print(f"Global best improved. Cost: {global_cost:.2f} Pos: {gbest}")

        return gbest, global_cost

    def step(self, gbest: tuple[float, float], signal_pos: tuple[float, float], global_cost: float,
             has_obstacle: Callable[[int, int], bool] = None, has_target: Callable[[int, int], bool] = None) -> float:
        self.update_velocity(gbest)
        self.update_position(has_obstacle, has_target)
        
        # signal_distance handler will be substituted by the sensor data in the future
        signal_distance = self.get_euclidean_distance(signal_pos, self.X)
        
        return self.update_pbest(signal_distance, global_cost, gbest)

    def draw(self, ax: plt.Axes):
        return ax.plot(self.X[0], self.X[1], color='purple', marker='o')[0]

# Simulation

In [52]:
class Simulation:
    def __init__(self, maze: Map, agents: list[Agent]) -> None:
        self.maze = maze
        self.agents = np.array(agents, dtype=object)
        self.points = [None] * self.agents.shape[0]
        self.global_cost = None
        self.gbest = np.array((0, 0))
        self.gbest_draw = None

        for i in range(self.agents.shape[0]):
            self.agents[i].id = i
    
    def shuffle_agents(self) -> None:
        self.agents = np.random.permutation(self.agents)
        self.points = [self.points[agent.id] for agent in self.agents]
    
    def step_all(self) -> None:
        self.shuffle_agents()

        for i, agent in enumerate(self.agents):
            self.points[i].set_data(agent.X[0], agent.X[1])
        
        for i, agent in enumerate(self.agents):
            self.gbest, self.global_cost = agent.step(self.gbest, self.maze.goal, self.global_cost)

    def step_one(self) -> None:
        index = np.random.randint(0, len(self.agents))

        if self.points[index] is not None:
            self.points[index].set_data(self.agents[index].X[0], self.agents[index].X[1])

        self.gbest, self.global_cost = self.agents[index].step(self.gbest, self.maze.goal, 
                                                               self.global_cost, self.maze.has_obstacle)

    def animate(self, i: int) -> None:
        
        # self.step_all()
        self.step_one()

        if self.gbest_draw is not None:
            self.gbest_draw.set_offsets(self.gbest)
        
        #TODO: Add stop condition
        # if self.global_cost < 0.01:
        #     print("Global cost below 0.01. Stopping simulation...")
        #     self.anim.pause()

        return self.points

    def draw(self, ax: plt.Axes) -> None:
        self.maze.draw(ax)
        for i, agent in enumerate(self.agents):
            point = agent.draw(ax)
            self.points[i] = point

        ax.scatter(self.maze.goal[0], self.maze.goal[1], color='red', marker='x')
        self.gbest_draw = ax.scatter(self.gbest[0], self.gbest[1], color='green', marker='x')

    def run(self, epochs: int = 300) -> None:
        fig, ax = plt.subplots()
        self.draw(ax)
        fig.tight_layout()
        print("Simulating...")
        self.anim = animation.FuncAnimation(
            fig,
            self.animate,
            frames=epochs,
            interval=100,
            blit=True
        )

        self.anim.save('animation.gif', fps=10)
        # HTML(anim.to_jshtml(fps=10))
        plt.show()

    def run_raw(self, epochs: int = 300) -> None:
        for i in range(epochs):
            self.animate(i)
            time.sleep(0.5)

    def display(self, figsize: tuple[int, int] = (10, 10)) -> None:
        fig, ax = plt.subplots(figsize=figsize)
        self.draw(ax)
        fig.tight_layout()


# Execution

In [54]:
LENGTH = 20
HEIGHT = 20
NUM_AGENTS = 10

maze_shape = (LENGTH, HEIGHT)
maze = Map(maze_shape)
maze.set_start(0, 0)
maze.set_goal(0, 19)
maze.set_square_wall(0, 5, 10, 2)
maze.set_square_wall(12, 15, 8, 2)

rand = np.random.rand

agents = [
    Agent([rand()*10, rand()*4], [rand()*2, rand()*2], weight=0.8, c1 = 0.5, c2 = 0.7, map_shape=maze_shape) 
    for i in range(NUM_AGENTS)
]

sim = Simulation(maze, agents)

sim.run(epochs=500)

<IPython.core.display.Javascript object>

MovieWriter ffmpeg unavailable; using Pillow instead.


Simulating...
Agent 9 hit an obstacle
Agent 1 hit an obstacle
Agent 7 hit an obstacle
Agent 0 hit an obstacle
Agent 1 hit an obstacle
Agent 3 hit an obstacle
Agent 4 hit an obstacle
Agent 6 hit an obstacle
Agent 3 hit an obstacle
Agent 2 hit an obstacle
Agent 8 hit an obstacle
Agent 0 hit an obstacle
Agent 0 hit an obstacle
Agent 5 hit an obstacle
Agent 8 hit an obstacle
Agent 4 hit an obstacle
Agent 2 hit an obstacle
Agent 8 hit an obstacle
Agent 9 hit an obstacle
Agent 3 hit an obstacle
Agent 2 hit an obstacle
Agent 3 hit an obstacle
Agent 3 hit an obstacle
Agent 1 hit an obstacle
Agent 6 hit an obstacle
Agent 8 hit an obstacle
Agent 5 hit an obstacle
Agent 6 hit an obstacle
Agent 7 hit an obstacle
Agent 5 hit an obstacle
Agent 6 hit an obstacle
Agent 4 hit an obstacle
Agent 9 hit an obstacle
Agent 7 hit an obstacle
Agent 0 hit an obstacle
Agent 2 hit an obstacle
Agent 1 hit an obstacle
Agent 7 hit an obstacle
Agent 1 hit an obstacle
Agent 0 hit an obstacle
Agent 4 hit an obstacle
Ag